# 미분 자동 계산


## 1 상자로서의 변수

### 1.1 변수란

변수란? 프로그래밍 입문서에서 변수는 상자라고 설명한다. 상자에 값을 담아두고 그 값을 사용할 수 있다.

- 상자와 데이터는 별개다
- 상자에는 데이터가 들어간다
- 상자 속을 들여다보면 데이터를 알 수 있다

### 1.2 Variable 클래스 구현

```python
class Variable:
    def __init__(self, data):
        self.data = data
```

**init** 메서드는 Variable 클래스의 생성자이다. 생성자는 인스턴스를 초기화하는 메서드이다.


In [1]:
import numpy as np


class Variable:
    def __init__(self, data):
        self.data = data


data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


머신러닝 시스템은 기본적으로 `다차원 배열`을 사용한다.


In [2]:
x.data = np.array(2.0)
print(x.data)

2.0


## 2 변수를 낳는 함수

### 2.1 함수란

어떤 변수로부터 다른 변수의 대응 관계를 정한 것 함수 $f(x) = x^{2}$이 있다고 할때 $y=f(x)$라고 하면 $y$와$x$의 관계가 함수 $f$에 의해 결정된다. 즉 함수 $f$에 의해 $y$는 $x$의 제곱이다 라는 관계가 성립한다.


### 2.2 Function 클래스 구현

- Function클래스는 Variable인스턴스를 입력받아 Variable인스턴스를 출력한다.
- Variable 인스턴스의 실제 데이터는 인스턴스 변수인 data에 저장된다.


In [3]:
class Function:
    def __call__(self, input):
        x = input.data
        y = x**2
        output = Variable(y)
        return output

`__call__`메서드의 인수 input은 Variable 인스턴스라고 가정한다. 따라서 실제 데이터는 input.data에 존재한다.
데이터를 꺼낸 후 원하는 계산을 하고 결과를 Variable 인스턴스로 되돌려준다.


In [4]:
x = Variable(np.array(10))
f = Function()
y = f(x)
print(type(y))
print(y.data)

<class '__main__.Variable'>
100


In [5]:
import numpy as np
from framework.variable import Variable
from framework.function import Square

x = Variable(np.array(10))
f = Square()
y = f(x)
print(type(y))
print(y.data)

<class 'framework.variable.Variable'>
100


## 3 함수 연결

### 3.1 Exp 함수 구현

$y=e^{x}$이는 계산을 하는 함수 $e$는 자연로그 밑**base of the natural logarithm**이다.


In [6]:
import numpy as np
from framework.function import Function


class Exp(Function):
    def forward(self, x):
        return np.exp(x)

## 3.2 함수 연결

Function 클래스의 **call** 메서드는 입력과 출력이 모두 Variable 인스턴스이다.


In [7]:
import numpy as np
from framework.function import Square, Exp
from framework.variable import Variable

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

print(y.data)

1.648721270700128


3개의 함수 A, B, C를 연이어 적용했다. 중간에 등장하는 4개의 변수 x, a, b, y가 모두 Variable 인스턴스이다. 이처럼 함수를 연결하여 사용할 수 있는 것은 모두 Function 클래스의 `__call__` 메서드가 Variable 인스턴스를 입력받고 Variable 인스턴스를 출력하기 때문이다.


## 4 수치 미분

`Variable`클래스와 `Function`클래스를 구현했다.

### 4.1 미분이란

미분이란 무엇인가? 미분은 `변화율`을 뜻한다. 예를 들어 물체의 시간에 따른 위치 변화율(위치의 미분)은 속도가 된다.
시간에 대한 속도 변화율(속도의 미분)은 가속도에 해당한다. 정확히는 `극한으로 짧은 시간(순간)에서의 변화량`이다.

$$ f'(x) = \lim\_{h \to 0} \frac{f(x+h) - f(x)}{h} $$

$\lim_{h \to 0}$은 $h$를 한없이 0에 가깝게 한다는 의미이다. 이를 수식으로 나타내면 $h$를 0으로 한없이 가깝게 한다는 것이다. $\frac{f(x+h) - f(x)}{h}$는 $x$에서의 $f$의 변화량을 나타낸다. 이 변화량을 $h$에 대한 함수로 나타낸 것이 $f'(x)$이다.


### 4.2 수치 미분 구현

컴퓨터는 극한을 표현할 수 없으므로 $(=1e-4)$과 같은 매우 작은 값을 $h$를 극한과 비슷한 값으로 대체한다. 이렇게 미세한 $h$값을 이용해 미분을 계산하는 방법을 `수치 미분`이라고 한다.

수치 미분은 작은 값을 사용하여 `진정한 미분`을 근사한다. 따라서 값에 어쩔 수 없는 오차가 포함된다. 이 근사 오차를 줄이는 방법으로 `중앙차분`이라는게 있다.
중앙 차분은 $f(x)$와 $f(x+h)$의 차이를 구하는 대신 $f(x-h)와 f(x+h)$의 차이를 구한다.


In [8]:
def numerical_diff(f, x, eps=1e-4):
    """수치 미분을 구하는 함수"""
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)

실체 데이터는 Variable의 인스턴수 변수인 data에 들어 있다.


In [9]:
from framework.variable import Variable
from framework.function import Square

f = Square()
x = Variable(np.array(2.0))
dy = numerical_diff(f, x)
print(x.data)
print(dy)

2.0
4.000000000004


### 4.3 합성 함수의 미분

$y=x^{2}$이라는 단순한 함수를 다루었다. 합성함수를 미분한다. $y=(e^{x^{2}})$이라는 계산에 대한 미분 $\frac{dy}{dx}$를 구해보자.


In [10]:
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))


x = Variable(np.array(0.5))
dy = numerical_diff(f, x)
print(dy)

3.2974426293330694


실행 결과를 보면 미분한 값이 3.297...이다. x를 0.5에서 작은 값만큼 변화시키면 y는 3.297...만큼 변화한다.


### 4.4 수치 미분의 문제점

수치 미분은 작은 오차를 가지고 있지만 계산에 따라 커질 수 있다. 또한 수치 미분은 계산량이 많다.


## 5 역전파 이론

역전파는 수치미분에 비해 오차량이 적다.

### 5.1 연쇄 법칙

연쇄 법칙 **chain rule**은 역전파를 이해는 열쇄이다. 밑바닥부터 시작하는 딥러닝 1,2,3,4권에서는 역전파를 설명한다. 그만큼 중요하다.

연쇄 법칙은 여러 함수를 사슬처럼 연결하여 사용하는 모습을 빗댄 이름이다. 연쇄 법칙에 따르면 함숭 함수(여러 함수가 연결된 함수)의 미분은 구성 함수 각각을 미분한 후 곱한 것과 같다.

$y=f(x)$라는 함수가 있고 이함수는 $a=A(x)$, $b=B(a), y=C(b)$와 같이 연결되어 있다고 하자. 이때 $y$를 $x$로 미분하면 다음과 같다.

$$
\frac{dy}{dx} = \frac{dy}{db} \frac{db}{da} \frac{da}{dx}
$$

$x$에 대한 $y$의 미분은 구성 함수 각각의 미분값을 모두 곱한 값과 같다.


**연쇄법칙**

- $x$에 대한 $y$의 미분은 구성 함수 각각의 미분값을 모두 곱한 값과 같다.
- 함수의 미분은 각 함수의 국소적인 미분들로 분해할 수 있다.

$$\frac{dy}{dx} = \frac{dy}{dy} \frac{dy}{db} \frac{db}{da} \frac{da}{dx}$$

$\frac{dy}{dy}$는 자신에 대한 미분이라 항상 1이다.


### 5.2 역전파의 원리 도출

연쇄 법칙대로 해석하면 합성함수의 미분은 구성 함둘들의 미분의 곱으로 분해할 수 있다. `곱하는 순서`는 중요하지 않다.
$$\frac{dy}{dx} = ((\frac{dy}{dy} \frac{dy}{db}) \frac{db}{da})\frac{da}{dx}$$


## 6 수동 역전파

### 6.1 Variable 클래스 추가 구현

통상값 {data}와 이에 대응하는 미분값 {grad}를 저장한다.


In [11]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None  # 미분값을 저장하는 변수

### 6.2 Function 클래스 추가 구현

- 미분을 계산하는 역전파(backward)를 추가한다.
- forward 메서드 호출시 건네받은 Variable 인스턴스를 기억해둔다.


In [12]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input  # 입력 변수를 기억(보관)한다.
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()

### 6.3 Square와 Exp 클래스 추가 구현

Square클래스는 $y=x^{2}$을 계산하는 클래스이다. 미분은 $\frac{dy}{dx}=2x$이다.


In [13]:
class Square(Function):
    def forward(self, x):
        y = x**2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx

Exp클래스는 $y=e^{x}$를 계산하는 클래스이다. 미분은 $\frac{dy}{dx}=e^{x}$이다.


In [14]:
import numpy as np


class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y

    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x) * gy
        return gx

### 6.4 역전파 구현


In [15]:
import numpy as np
from framework.variable import Variable
from framework.function import Square, Exp

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)  # $$\frac{dy}{dy} = 1$$ 이기 때문에 1.0부터 시작한다.
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


## 7. 역전파 자동화


In [16]:
import numpy as np
from framework.variable import Variable
from framework.function import Square, Exp

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

In [17]:
y.grad = np.array(1.0)
C = y.creator
b = C.input
b.grad = C.backward(y.grad)

In [18]:
B = b.creator
a = B.input
a.grad = B.backward(b.grad)

In [19]:
A = a.creator
x = A.input
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


In [20]:
import numpy as np
from framework.variable import Variable
from framework.function import Square, Exp

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


## 8 재귀에서 반복문으로

```python
class Variable:
  ...
  def backward(self):
    functions = [self.creator]
    while functions:
      f = functions.pop()
      x, y = f.input, f.output
      x.grad = f.backward(y.grad)

      if x.creator is not None:
        functions.append(x.creator)
```


In [21]:
import numpy as np
from framework.variable import Variable
from framework.function import Square, Exp

A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256
